In [ ]:
import mediapipe as mp
import cv2 
import os
import numpy as np   
import imgaug.augmenters as iaa

# Define a sequence of augmentations

augmentations = iaa.Sequential([
    iaa.Fliplr(0.5),            
    iaa.Affine(
        rotate=(-20, 20),        
        scale=(0.9, 1.1)         
    ),
    iaa.Multiply((0.8, 1.2)),    
    iaa.GaussianBlur(sigma=(0, 1.0))  
])


# Initialize Mediapipe Pose model

mp_pose = mp.solutions.pose            
pose = mp_pose.Pose()                  

# Function to extract 33 pose landmarks from one image

def extract_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.pose_landmarks:
        return np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten()
    else:
        return None
    
dataset_path = 'yoga_dataset'

X = []  # landmarks
y = []  # Class labels

# Loop through pose folders

for pose_label in os.listdir(dataset_path):       
    pose_folder = os.path.join(dataset_path, pose_label)
    if not os.path.isdir(pose_folder):
        continue

    for image_file in os.listdir(pose_folder):
        image_path = os.path.join(pose_folder, image_file)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Image not found: {image_path}")
            continue

        # Original image landmarks 

        landmarks = extract_landmarks(image_path)
        if landmarks is not None:
            X.append(landmarks)
            y.append(pose_label)

        # Apply N augmentations per image
        
        N = 3  
        for _ in range(N):
            aug_image = augmentations(image=image)
            aug_results = pose.process(cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB))
            if aug_results.pose_landmarks:
                aug_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in aug_results.pose_landmarks.landmark]).flatten()
                X.append(aug_landmarks)
                y.append(pose_label)


print(f"Total samples collected: {len(X)}")
print("Detected pose labels:", set(y))


In [3]:
X = np.array(X)  

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.971830985915493
               precision    recall  f1-score   support

   Chakrasana       1.00      0.95      0.98        22
  Dhanurasana       0.92      1.00      0.96        11
Padahastasana       0.92      1.00      0.96        11
  Trikonasana       1.00      1.00      1.00        11
  Vrikshasana       1.00      0.94      0.97        16

     accuracy                           0.97        71
    macro avg       0.97      0.98      0.97        71
 weighted avg       0.97      0.97      0.97        71



In [5]:
import joblib

# Save the model to a file
joblib.dump(model, 'yoga_pose_classifier.pkl') 

# Save the label encoder to a file
joblib.dump(y, 'pose_label_encoder.pkl')

print("Model and label encoder saved successfully.")


Model and label encoder saved successfully.


In [ ]:


# Load trained model and label encoder
model = joblib.load('yoga_pose_classifier.pkl')
label_encoder = joblib.load('pose_label_encoder.pkl')
                                                            
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()
                                                                      
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
                                                                  
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract landmark data

        landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten().reshape(1, -1)

        # To predict pose
        
        probs = model.predict_proba(landmarks)[0]
        max_prob = np.max(probs)
        pred_class = np.argmax(probs)

        pose_name = model.classes_[pred_class] if max_prob > 0.70 else "Unknown"

        # To display predicted pose

        cv2.putText(frame, f'Pose: {pose_name}', (30, 50),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Yoga Pose Detection', frame)

    # Press 'q' to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
